In [1]:
# ✅ Environment setup
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

# ✅ Install dependencies
!pip install transformers datasets evaluate accelerate --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 10.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import torch


2025-11-01 09:18:18.108293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761988698.498005      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761988698.608265      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# ✅ Load the dataset (first 100K rows)
import pandas as pd

df = pd.read_csv("/kaggle/input/task1-ibi/data_100k.csv")

# ✅ Basic inspection
print("✅ Dataset loaded successfully!")
print("Shape:", df.shape)
display(df.head())

# ✅ Check for null values
print("\nMissing values per column:")
print(df.isnull().sum())


✅ Dataset loaded successfully!
Shape: (100000, 6)


,title,NER,Extended_NER,genre,label,directions
0,\t Arugula Pomegranate Salad,"[""baby spinach"", ""baby arugula"", ""pomegranate ...","['alfalfa sprouts', 'baby spinach', 'baby arug...",vegetables,4,"[""Toss together spinach and arugula, then plac..."
1,\t Black Bean And Turkey Chili,"[""olive oil"", ""yellow onion"", ""garlic"", ""groun...","['one', 'yellow onion', 'tomato paste', 'about...",sides,8,"[""Dice the onion and mince the garlic. Add the..."
2,\t Finger Lickin' Tofu Nuggets,"[""extra firm"", ""almond flour"", ""nutritional ye...","['extra firm', '2', 'coconut oil', 'almond flo...",nonveg,3,"[""Wrap the tofu in a clean tea towel and press..."
3,\t Jerk Beef Stew With Carrots And Tomatoes,"[""olive oil"", ""boneless beef chuck"", ""onion"", ...","['boneless beef chuck', '2', 'Saute', 'onion',...",vegetables,4,"[""Preheat oven to 350 degrees F."", ""Heat the o..."
4,\t Pomegranate Couscous Salad,"[""pomegranate arils"", ""whole wheat couscous"", ...","['whole wheat couscous', '10 minutes', 'lemon ...",vegetables,4,"[""Place couscous in a bowl with 11/2 cups of h..."



Missing values per column:
title           0
NER             0
Extended_NER    0
genre           0
label           0
directions      0
dtype: int64


In [4]:
def make_prompt(row):
    title = str(row['title'])
    ingredients = str(row['Extended_NER'])  # use correct column name
    instructions = str(row['directions'])
    return f"TITLE: {title}\nINGREDIENTS: {ingredients}\nRECIPE: {instructions}"

# Create a new text column for training
df['text'] = df.apply(make_prompt, axis=1)

# Keep only the text column for GPT-2 training
df = df[['text']]

df.head(3)


,text
0,TITLE: \t Arugula Pomegranate Salad\nINGREDIEN...
1,TITLE: \t Black Bean And Turkey Chili\nINGREDI...
2,TITLE: \t Finger Lickin' Tofu Nuggets\nINGREDI...


In [5]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

train_ds = dataset['train']
test_ds = dataset['test']


In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # add padding token


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
def tokenize_fn(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_ds = train_ds.map(tokenize_fn, batched=True, remove_columns=["text"])
test_ds = test_ds.map(tokenize_fn, batched=True, remove_columns=["text"])


Map:   0%|          | 0/90000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-recipes",
    # Remove evaluation_strategy if your version doesn't support it
    # evaluation_strategy="epoch",  # uncomment if you upgraded transformers
    per_device_train_batch_size=2,
    num_train_epochs=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    fp16=True,
    logging_steps=100,
    save_total_limit=2,
    save_strategy="epoch",
        report_to=[]  # <- important! disables all integrations like wandb

)


In [9]:
trainer = Trainer(
    model=AutoModelForCausalLM.from_pretrained("gpt2"),
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    processing_class=tokenizer,  # future-safe replacement
    data_collator=DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
),
)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.277000
200,1.161900
300,1.129100
400,1.060300
500,1.083700
600,1.053300
700,1.032100
800,1.053200
900,1.033700
1000,1.032600


TrainOutput(global_step=5625, training_loss=0.9868319390190973, metrics={'train_runtime': 4935.3641, 'train_samples_per_second': 18.236, 'train_steps_per_second': 1.14, 'total_flos': 1.5370801818624e+16, 'train_loss': 0.9868319390190973, 'epoch': 1.0})

In [11]:
trainer.save_model("./gpt2-recipes-final")
tokenizer.save_pretrained("./gpt2-recipes-final")


('./gpt2-recipes-final/tokenizer_config.json',
 './gpt2-recipes-final/special_tokens_map.json',
 './gpt2-recipes-final/vocab.json',
 './gpt2-recipes-final/merges.txt',
 './gpt2-recipes-final/added_tokens.json',
 './gpt2-recipes-final/tokenizer.json')

In [14]:
from transformers import pipeline

generator = pipeline(
    "text-generation", 
    model="./gpt2-recipes-final", 
    tokenizer="./gpt2-recipes-final"
)

prompt = "TITLE: Simple Pancakes\nINGREDIENTS: eggs, milk, flour, sugar\nRECIPE:"
outputs = generator(prompt, max_length=200, num_return_sequences=1, do_sample=True, top_k=50, top_p=0.95)

print(outputs[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


TITLE: Simple Pancakes
INGREDIENTS: eggs, milk, flour, sugar
RECIPE: ["Beat eggs, milk, flour, sugar, and spices together.", "Stir in pancake mixture.", "Pour into greased baking dish.", "Bake at 350\u00b0 for 1 hour."]
RECIPE: ["Preheat oven to 350\u00b0.", "Melt butter in a large skillet over medium heat.", "Add eggs and cook over medium heat, stirring constantly until golden brown, about 1 minute."]
Serve with crackers or crackers cake.", "Serves 6."]
RECIPE: ["In a small bowl, whisk together egg yolks and oil. Add flour, salt and pepper.", "Mix well.", "Add pancake mixture, stirring to combine.", "Sprinkle with crackers.", "Top with remaining batter."]
RECIPE: ["In a large skillet over medium heat, melt butter.", "Add eggs, then add milk, stirring to combine.", "Pour into a greased baking dish.", "Bake at 350\u00b0 for 1 hour."]"]]"]]"]]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]"]
